<a href="https://colab.research.google.com/github/Seferovic8/chess-project/blob/main/models/Dense_Chess_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=f4e38f530d56bfb8d98e9fd1473da259c74f547ccb16b35304101d502f7a451a
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess


In [ ]:
import numpy as np
import pandas as pd
import torch
from numpy import loadtxt
from io import StringIO
import chess.pgn
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from datetime import datetime


In [ ]:
#!gdown 1ruGgJKeKv2DGWgVcqmdjK9pWk0mPYSu4
!gdown 1OjVvEeEZNTvU6M2s2We6u8pFYil3dS1x

Downloading...
From (original): https://drive.google.com/uc?id=1OjVvEeEZNTvU6M2s2We6u8pFYil3dS1x
From (redirected): https://drive.google.com/uc?id=1OjVvEeEZNTvU6M2s2We6u8pFYil3dS1x&confirm=t&uuid=3f3d78a2-382d-47e1-89b7-231eb5f7f4ad
To: /content/chess_dataset.csv
100% 2.73G/2.73G [00:17<00:00, 152MB/s]


In [ ]:
def string_to_array(string):
    string = string.replace("]", "").replace("[", "").replace("\n","").replace(" ","").replace("."," ")
    #k = '['+k+']'
    c = StringIO(string)

    ar=loadtxt(c)
    return ar.reshape((8,8,12))

In [ ]:
df = pd.read_csv("chess_dataset.csv")
#df = df[:500_000]

In [ ]:
len(df)

1056507

In [ ]:
df["board"]=df["board"].apply(string_to_array)

In [ ]:
df

,board,fromPosition,toPosition,player
0,"[[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",50,34,0
1,"[[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",51,43,0
2,"[[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",34,27,0
3,"[[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",57,42,0
4,"[[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",62,45,0
...,...,...,...,...
1056502,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",51,44,0
1056503,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",25,49,0
1056504,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",43,34,0
1056505,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",34,25,0


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2)

In [ ]:
def one_hot_encode(num, length=64):
    enc = np.zeros(length)
    enc[num]=1
    return enc

In [ ]:
class ChessDataset(Dataset):
  def __init__(self, dataset):
    super(ChessDataset,self).__init__()
    self.dataset = dataset

  def __len__(self):
      return len(self.dataset)

  def __getitem__(self, idx):
    # get audio data
    row = df.loc[idx]
    board=row["board"]
    from_position = row['fromPosition']
    player = row['player']
    # Convert to torch tensors with appropriate data types
    board_tensor = torch.FloatTensor(board)
    from_position_tensor = torch.tensor(one_hot_encode(from_position), dtype=torch.float32)
    player_tensor = torch.tensor(player, dtype=torch.float32)
    # Get label
    label = torch.tensor(one_hot_encode(row['toPosition']), dtype=torch.float32)

    # Return tuple of inputs and label

    return (board_tensor, from_position_tensor, player_tensor), label

In [ ]:
train_dataset = ChessDataset(df_train)
test_dataset = ChessDataset(df_test)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
for i,_ in train_loader:
  print(i[2].shape)
  break

torch.Size([32])


In [ ]:
for i,d in train_dataset:
  print(i[2].view(-1,1).shape)
  break

torch.Size([1, 1])


In [ ]:
number_of_classes=64

# Create model

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
bada=[]

In [ ]:
class ChessDecoder:
      def __init__(self):
          self.pieces={
          chess.Piece(chess.PAWN, chess.WHITE): 0,
          chess.Piece(chess.PAWN, chess.BLACK): 1,
          chess.Piece(chess.ROOK, chess.WHITE): 2,
          chess.Piece(chess.ROOK, chess.BLACK): 3,
          chess.Piece(chess.KNIGHT, chess.WHITE): 4,
          chess.Piece(chess.KNIGHT, chess.BLACK): 5,
          chess.Piece(chess.BISHOP, chess.WHITE): 6,
          chess.Piece(chess.BISHOP, chess.BLACK): 7,
          chess.Piece(chess.QUEEN, chess.WHITE): 8,
          chess.Piece(chess.QUEEN, chess.BLACK): 9,
          chess.Piece(chess.KING, chess.WHITE): 10,
          chess.Piece(chess.KING, chess.BLACK): 11,
        }
          self.reverse_pieces = {value: key for key, value in self.pieces.items()}
      def getPiece(self,encoding):
          if torch.equal(encoding, torch.zeros(12).to(device)):
            return None

          return self.reverse_pieces[torch.argmax(encoding).item()]

      def getPieceEncoding(self,piece):
          if piece is not None:
              return one_hot_encode(self.pieces[piece],length=12)
          return torch.zeros(12)

      def decodeBoardPosition(self, encodedBoard,turn):
          encodedBoard=encodedBoard.reshape(64,-1)
          board=chess.Board()
          board.turn=turn
          for square in range(64):
              board.set_piece_at(square,self.getPiece(encodedBoard[square]))
          return board
      def getAllowedMoves(self,b,square,winner,en_passant=False):
              global bada
              board = self.decodeBoardPosition(b,winner)
              print(type(board))
              isLeftEdge=square%8!=0
              isRightEdge=square%8!=7
              if en_passant:
                if winner ==chess.BLACK:
                    if isLeftEdge:
                      board.ep_square = square-9
                    if isRightEdge:
                      board.ep_square = square-7
                else:
                    if isRightEdge:
                      board.ep_square = square+9
                    if isLeftEdge:
                      board.ep_square = square+7
              allowedMoves = [move.to_square for move in board.pseudo_legal_moves if move.from_square == square]
              if not allowedMoves:
                return self.getAllowedMoves(b,square,winner,en_passant=True)
              return allowedMoves

      def getMask(self,encodedBoard,from_square,player,mask_size):
          mask = torch.zeros(mask_size, 64).to(device)  # Initialize mask with zeros
          #mask.fill_(-10)

          for i,board in enumerate(encodedBoard):
              winner = bool(player[i].item())
              square=torch.argmax(from_square[i]).item()

              allowedMoves=[]
              allowedMoves = self.getAllowedMoves(board,square,winner)
             # mask[i, allowedMoves] = 0
              mask[i, allowedMoves] = 1
          return mask
      def encodeBoard(self,board):
        encodedBoard=np.zeros((64,12))
        for square in range(64):
            piece = board.piece_at(square)
            encodedBoard[square]=self.getPieceEncoding(piece)
        return encodedBoard.reshape(8,8,12)

      def getWinner(game):
        if game.headers['Result'] =="1-0":
            return chess.WHITE
        return chess.BLACK

      def encodeGame(self,game):
        board = chess.Board()
        data=[]
        winner = self.getWinner(game)
        # Go through each move in the game until
        # we reach the required move number
        for move in game.mainline_moves():
            if board.turn==winner:
                encodedBoard=self.encodeBoard(board)
                data.append([encodedBoard,move.from_square,move.to_square,int(winner)])
            board.push(move)
        return data

In [ ]:
class CNN(nn.Module):
  def __init__(self, n_inputs, n_outputs):
    super(CNN,self).__init__()
    self.D = n_inputs
    self.K = n_outputs
    self.conv1 = nn.Sequential(
      nn.Conv2d(self.D, 32, kernel_size=5, padding=2,stride=1),
      nn.ReLU(),
      nn.Conv2d(32, 64, kernel_size=3,padding=1,stride=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      #nn.MaxPool2d(2,stride=2)
    )

    self.conv2 = nn.Sequential(
      nn.Conv2d(64, 128, kernel_size=3,padding=3,stride=1),
      nn.ReLU(),
      nn.Conv2d(128, 256, kernel_size=3,padding=2,stride=1),
      nn.BatchNorm2d(256),
      nn.ReLU(),
     #nn.MaxPool2d(2)
    )

    self.conv3 = nn.Sequential(
      nn.Conv2d(256, 512, kernel_size=2,padding=0,stride=1),
      nn.BatchNorm2d(512),
      nn.ReLU(),
     # nn.MaxPool2d(2)
    )
    self.conv4 = nn.Sequential(
      nn.Conv2d(512, 512, kernel_size=2,padding=0,stride=1),
      nn.BatchNorm2d(512),
      nn.ReLU(),
      #nn.MaxPool2d(2)
    )
    self.fc = nn.Sequential(
      nn.Dropout(0.5),
      nn.Linear(73728,10240),
      nn.ReLU(),
      nn.Linear(10240, 5120),
      #nn.ReLU(),
      nn.Linear(5120, 2056),
      nn.ReLU(),
      nn.Linear(2056, self.K),
      nn.ReLU()
    )

  def forward(self,X):
   # X = torch.unsqueeze(X,1)

    out = self.conv1(X)

    out = self.conv2(out)

    out = self.conv3(out)

    out = self.conv4(out)
    #out = self.conv5(out)

    #out, _  = torch.max(out, -1)
    out = out.view(out.size(0), -1)
    out = self.fc(out)
    return out



In [ ]:
class ChessModel(nn.Module):
  def __init__(self, hidden_outputs, n_outputs,device):
    super(ChessModel,self).__init__()

    self.HO=hidden_outputs
    self.K = n_outputs
    self.device=device
    #self.cnn = CNN(n_inputs=12, n_outputs=self.HO)
    self.chessDecoder=ChessDecoder()
    self.fcPosition = nn.Sequential(
        nn.Linear(64,128),
        nn.ReLU(),
        nn.Linear(128,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU())
    self.fcPlayer = nn.Sequential(
        nn.Linear(1,64),
        nn.ReLU(),
        nn.Linear(64,64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.ReLU())
    self.fc = nn.Sequential(
        nn.Linear(768+64+32,4096),
        nn.ReLU(),
        nn.Linear(4096,4096),
        nn.ReLU(),
        nn.Linear(4096,2048),
        nn.ReLU(),
        nn.Linear(2048,self.K))

  def forward(self,X):
    #cnn_outputs = self.cnn(X[0])
    position_outputs = self.fcPosition(X[1])
    player_outputs = self.fcPlayer(X[2])
    out = torch.cat((X[0].view(X[0].size(0),-1),position_outputs,player_outputs),1)
   # out = torch.cat((cnn_outputs,position_outputs,palyer_outputs),1)
    out = self.fc(out)
    #out = torch.softmax(out,dim=1)
   # mask=self.chessDecoder.getMask(X[0],X[1],X[2],out.size(0))
    #out=out*mask
    #out = out / out.sum(dim=1, keepdim=True)
    return out

In [ ]:
model = ChessModel(
            hidden_outputs = 512,
            n_outputs=number_of_classes,
            device=device)
model.to(device)

ChessModel(
  (fcPosition): Sequential(
    (0): Linear(in_features=64, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
  )
  (fcPlayer): Sequential(
    (0): Linear(in_features=1, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=864, out_features=4096, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4096, out_features=4096, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4096, out_features=2048, bias=True)
    (5): ReLU()
    (6): Linear(in_features=2048, out_features=64, bias=True)
  )
)

In [ ]:
criterion = nn.CrossEntropyLoss()
chessDecoder=ChessDecoder()
l=0
for i, k in train_loader:
 # print(i[0].shape)
 # i = i.view(i.size(0),1,i.size(1),i.size(2))
  l+=1
  i = i[0].reshape((-1,12,8,8)).to(device),i[1].to(device),i[2].view(-1,1).to(device)
  #print(k[0][34])
  out = model(i)
  if np.isnan(criterion(out,k.to(device)).item()):
    print(out[0])

  #print(criterion(out,k.to(device)).item())
  #print(type(criterion(out,k.to(device)).item()))
  break
  #print(np.max(lak[0].cpu().detach().numpy()))
  #print(lak.size())


# Train model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

In [ ]:
def save_model_checkpoint(val_loss, best_val_loss, train_loss, it, model=model, optimizer=optimizer,path='model_checkpoint.pt'):
    torch.save({
            'epoch': it,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            }, path)
    print(f'Model saved at epoch: {it+1}, val_loss improved from: {best_val_loss:.4f} to: {val_loss:.4f}')

In [ ]:
def plot_loss_curves(train_losses,val_losses,train_losses2=None,val_losses2=None):
  if(train_losses2 is not None):
    train_losses = np.concatenate((train_losses,train_losses2))
    val_losses = np.concatenate((val_losses,val_losses2))
  plt.plot(train_losses,label='Train loss')
  plt.plot(val_losses,label='Val loss')
  plt.legend(); plt.show
best_val_loss=10**5

In [ ]:
epoches=50
train_losses = np.zeros(epoches)
val_losses = np.zeros(epoches)

for it in range(epoches):
  t0 = datetime.now()
  model.train() # set model to train mode
  train_loss=[]
  val_loss=[]

  # train
  for inputs,targets in train_loader:
    # move data to gpu
    inputs= inputs[0].reshape((-1,12,8,8)).to(device),inputs[1].to(device),inputs[2].view(-1,1).to(device)

    targets = targets.to(device)
    #inputs = inputs.permute(0,2,1)
    # zero gradients
    optimizer.zero_grad()

    # forward pass
    outputs = model(inputs)
    loss = criterion(outputs,targets)

    # backward
    loss.backward()
    optimizer.step()

    train_loss.append(loss.item())
  model.eval() # set model to eval mode
  for inputs,targets in test_loader:
    # move data to gpu
    inputs= inputs[0].reshape((-1,12,8,8)).to(device),inputs[1].to(device),inputs[2].view(-1,1).to(device)
    targets = targets.to(device)
   # inputs = inputs.permute(0,2,1)


    # forward pass
    outputs = model(inputs)
    loss = criterion(outputs,targets)

    val_loss.append(loss.item())

  # calculate loss
  train_loss = np.mean(train_loss)
  val_loss = np.mean(val_loss)
  if (val_loss<best_val_loss):
    save_model_checkpoint(val_loss,best_val_loss,train_loss,it, path='model_checkpoint.pt')
    best_val_loss=val_loss
  # append loss
  train_losses[it]=train_loss
  val_losses[it]=val_loss
  dt = datetime.now() - t0
  print(f"Epoch {it+1}/{epoches}, Train loss: {train_loss:.4f}, Val loss: {val_loss:.4f}, Duration: {dt}")
  print('------------------------------------------------------------------------')

Model saved at epoch: 1, val_loss improved from: 100000.0000 to: 1.6206
Epoch 1/50, Train loss: 2.1438, Val loss: 1.6206, Duration: 0:04:50.938335
-------------------------------------------------------------
Model saved at epoch: 2, val_loss improved from: 1.6206 to: 1.2380
Epoch 2/50, Train loss: 1.4221, Val loss: 1.2380, Duration: 0:04:49.970269
-------------------------------------------------------------
Model saved at epoch: 3, val_loss improved from: 1.2380 to: 0.9965
Epoch 3/50, Train loss: 1.1061, Val loss: 0.9965, Duration: 0:04:52.814412
-------------------------------------------------------------
Model saved at epoch: 4, val_loss improved from: 0.9965 to: 0.7982
Epoch 4/50, Train loss: 0.8463, Val loss: 0.7982, Duration: 0:04:48.400356
-------------------------------------------------------------
Model saved at epoch: 5, val_loss improved from: 0.7982 to: 0.6233
Epoch 5/50, Train loss: 0.6017, Val loss: 0.6233, Duration: 0:04:47.413008
-------------------------------------

KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load("model_checkpoint.pt")["model_state_dict"])


<ipython-input-35-309bddacb415>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_checkpoint.pt")["model_state_dict"])


<All keys matched successfully>

In [ ]:
model.eval() # set model to eval mode
val_loss=[]
for inputs,targets in test_loader:
    # move data to gpu
    inputs= inputs[0].reshape((-1,12,8,8)).to(device),inputs[1].to(device),inputs[2].view(-1,1).to(device)
    targets = targets.to(device)
   # inputs = inputs.permute(0,2,1)


    # forward pass
    outputs = model(inputs)
    loss = criterion(outputs,targets)

    val_loss.append(loss.item())

  # calculate loss
val_loss = np.mean(val_loss)

In [ ]:
val_loss

0.0870613561711931

In [ ]:
torch.save(model.state_dict(), 'model_to_weights.pth')